<!-- @format -->

Importing stuff


In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
import random
from torch.nn import functional as F


model_name = "flax-community/papuGaPT2"

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = AutoModelForCausalLM.from_pretrained(model_name).to(device)
tokenizer = AutoTokenizer.from_pretrained(model_name)

device

device(type='cuda')

<!-- @format -->

### Defining Trie, best_k, sample_from_pairs


In [16]:
class Trie:
    is_end: bool
    children: dict[str, "Trie"]

    def __init__(self):
        self.is_end = False
        self.children = {}

    def search(self, s):
        node = self
        for ch in s:
            if ch not in node.children:
                return None
            node = node.children[ch]
        return node if node.is_end else None

    def serch_by_letter(self, s, letter):
        node = self.search(letter)
        if node is None:
            return None
        return self.search(s)

    def insert(self, s):
        node = self.search(s)
        if node is None:
            node = self
            for ch in s:
                if ch not in node.children:
                    node.children[ch] = Trie()
                node = node.children[ch]
            node.is_end = True

    def print_all_nodes(self, prefix=""):
        if self.is_end:
            print(prefix)
        for head, child in self.children.items():
            child.print_all_nodes(prefix + head)


trie = Trie()
for token in tokenizer.get_vocab().keys():
    trie.insert(token)

In [17]:
def best_k(prefix, K=10):
    input_ids = tokenizer(prefix, return_tensors="pt")["input_ids"].to(device)
    output = model(input_ids=input_ids)
    next_token_logits = output.logits[0, -1, :]
    probs = F.softmax(next_token_logits, dim=-1)
    d = {}
    for i in range(probs.shape[0]):
        token = tokenizer.decode(i)
        d[token] = probs[i]
        if not trie.search(token):
            trie.insert(token)
    return [
        (t, d[t])
        for t in sorted(d, key=d.get, reverse=True)[:K]
        if trie.search_by_letter(t, prefix[0])
    ]


def sample_from_pairs(pairs):
    tokens = [p[0] for p in pairs]
    weights = [p[1] for p in pairs]
    return random.choices(tokens, weights=weights, k=1)[0]

<!-- @format -->

### Test


In [ ]:
# trie = Trie()
# trie.insert("h")
# trie.insert("hello")



# trie.insert("world")
# trie.insert("high")
# trie.insert("hi")
# # x = trie.search("hi")
# trie.print_all_nodes()
# print(trie.gets_all_words_from_branch("h"))

<!-- @format -->

```python=
Inicjalizuj drzewo trie
Ustaw prefiks jako początek zdania

Dla każdej iteracji (do maksymalnej długości zdania):
    Uzyskaj najlepsze tokeny za pomocą funkcji best_k(zdanie, K)
    Dla każdego tokena w najlepszych tokenach:
        Dodaj token do drzewa trie

    Filtrowanie tokenów w drzewie trie, aby zachować tylko te, które zaczynają się od tej samej litery co prefiks
    Jeśli nie ma żadnych tokenów po filtrowaniu:
        Przerwij pętlę

    Losowo wybierz jeden token z filtrowanych tokenów za pomocą funkcji sample_from_pairs(filtrowane tokeny)
    Dodaj wybrany token do zdania

    Jeśli wybrany token jest znakiem interpunkcyjnym kończącym zdanie:
        Przerwij pętlę

Zwróć wygenerowane zdanie
```


In [7]:
def load_prefixes():
    with open("prefiksy.txt", "r", encoding="utf-8") as read_file:
        return [line.rstrip() for line in read_file]


prefixes = load_prefixes()
rand_prefix = random.choice(prefixes)
rand_prefix

'Do dnia dzisiejszego'

In [8]:
def sample_demo(N, txt):
    for i in range(N):
        d = best_k(txt)
        print(txt)
        next_token = sample_from_pairs(d)
        for t, p in best_k(txt):
            star = ""
            if t == next_token:
                star = "*"
            print(f"   [{t}]{star} {p:.4f}")
        txt += next_token
        print()


sample_demo(4, rand_prefix)

Do dnia dzisiejszego
   [ nie] 0.0905
   [,] 0.0494
   [ w] 0.0402
   [ na] 0.0221
   [.] 0.0160
   [ jest] 0.0133
   [ (] 0.0129
   [ udało] 0.0122
   [ nikt]* 0.0088
   [ mam] 0.0085

Do dnia dzisiejszego nikt
   [ nie]* 0.5821
   [ z] 0.0801
   [ już] 0.0524
   [ się] 0.0433
   [ w] 0.0226
   [ mnie] 0.0146
   [ jeszcze] 0.0140
   [ chyba] 0.0118
   [ o] 0.0100
   [ nic] 0.0086

Do dnia dzisiejszego nikt nie
   [ wie] 0.0518
   [ jest] 0.0392
   [ ma]* 0.0346
   [ został] 0.0280
   [ może] 0.0206
   [ odpowiedział] 0.0178
   [ podjął] 0.0178
   [ miał] 0.0170
   [ był] 0.0155
   [ potrafi] 0.0143

Do dnia dzisiejszego nikt nie ma
   [ wątpliwości] 0.3757
   [ pewności] 0.0561
   [ pojęcia] 0.0329
   [ żadnych]* 0.0324
   [ zastrzeżeń] 0.0221
   [ już] 0.0207
   [ w] 0.0157
   [ nic] 0.0155
   [ do] 0.0140
   [ prawa] 0.0130

